In [ ]:
import pandas as pd
import numpy as np

# Parameters for the priority function
alpha = 1.0  # Weight for execution time
beta = 0.5   # Weight for resource demand
gamma = 0.3  # Weight for deadline proximity

# Parameters for the preemption threshold
lambda_threshold = 50  # Maximum allowable load threshold in %

# Parameters for migration overhead
delta = 0.2  # Coefficient for resource demand
epsilon = 0.1  # Coefficient for task size

# Network bandwidth and disk throughput (in MB/s)
network_bandwidth = 100  # Example value
disk_throughput = 200  # Example value

# Function to calculate priority score (Equation 1)
def calculate_priority(task_row):
    ti = task_row['Execution_Time(ms)']
    Ri = task_row['CPU_Requirement'] + task_row['Memory_Requirement(MB)'] + task_row['IO_Requirement(MB/s)']
    Di = task_row['Deadline(ms)']
    priority_score = (alpha / ti) + (beta * Ri) - (gamma * (Di - ti))
    print(f"Task {task_row['Task_ID']} - Priority Calculation: (α/ti) = {alpha/ti}, (β⋅Ri) = {beta*Ri}, -(γ⋅(Di−ti)) = {-gamma*(Di-ti)} -> Priority Score = {priority_score}")
    return priority_score

# Function to calculate preemption threshold (Equation 2)
def check_preemption_threshold(vm_row):
    LVMj = (vm_row['Current_Load_CPU(%)'] + vm_row['Current_Load_Memory(%)'] + vm_row['Current_Load_IO(%)']) / 3
    print(f"VM {vm_row['VM_ID']} - Checking preemption threshold: LVMj = {LVMj}, λ = {lambda_threshold} -> Preemption Required: {LVMj > lambda_threshold}")
    return LVMj > lambda_threshold

# Function to calculate migration overhead (Equation 3)
def calculate_migration_overhead(task_row):
    Ri = task_row['CPU_Requirement'] + task_row['Memory_Requirement(MB)'] + task_row['IO_Requirement(MB/s)']
    task_size = task_row['Memory_Requirement(MB)']  # Assume task size is proportional to memory requirement
    overhead = (delta * Ri) / network_bandwidth + (epsilon * task_size) / disk_throughput
    print(f"Task {task_row['Task_ID']} - Migration Overhead Calculation: Overhead = {overhead}")
    return overhead

# Read the  task and VM data
task_data = pd.read_csv('task_data.csv')
vm_data = pd.read_csv('vm_data.csv')

# Calculate dynamic priority scores for each task
print("\n### Calculating Priority Scores for All Tasks ###")
task_data['Priority_Score'] = task_data.apply(calculate_priority, axis=1)

# List to store tasks that need to be migrated
tasks_to_migrate = []

print("\n### Checking VMs for Preemption ###")
# Check for each VM if the load exceeds the preemption threshold
for vm_id, vm_row in vm_data.iterrows():
    if check_preemption_threshold(vm_row):
        print(f"\nVM {vm_row['VM_ID']} is overloaded. Analyzing tasks for migration...")

        # Get the tasks currently assigned to this VM
        tasks_on_vm = task_data[task_data['Assigned_VM_ID'] == vm_row['VM_ID']]

        # Sort tasks by priority score (lowest to highest)
        tasks_on_vm = tasks_on_vm.sort_values(by='Priority_Score')
        print(f"Sorted tasks for VM {vm_row['VM_ID']} based on priority:\n{tasks_on_vm[['Task_ID', 'Priority_Score']].to_string(index=False)}")

        # Check each task for potential migration
        for _, task_row in tasks_on_vm.iterrows():
            migration_overhead = calculate_migration_overhead(task_row)
            # Only migrate if the migration overhead is acceptable
            if migration_overhead < 1.0:  # Example threshold for migration overhead
                print(f"Task {task_row['Task_ID']} - Migration overhead acceptable. Task will be migrated.")
                tasks_to_migrate.append(task_row['Task_ID'])
            else:
                print(f"Task {task_row['Task_ID']} - Migration overhead too high. Task will not be migrated.")
                break  # Stop migrating tasks if overhead is too high

# Output the list of tasks that need to be migrated
print("\n### Tasks that need to be migrated based on priority analysis ###")
print(tasks_to_migrate)

# If needed, save the output to a file
with open('tasks_to_migrate.csv', 'w') as f:
    f.write("Task_ID\n")
    for task_id in tasks_to_migrate:
        f.write(f"{task_id}\n")


In [ ]:
!pip install PySDNSim

from PySDNSim.Backend import Backend
from PySDNSim.Config import Config
from PySDNSim.Experiment import Experiment
from PySDNSim.Host import Host
from PySDNSim.Job import Job
from PySDNSim.Microservice import Microservice
from PySDNSim.NetworkService import NetworkService, create_network_service

import numpy as np
import pandas as pd
from numpy.random import rand, randint, uniform
from sklearn.preprocessing import MinMaxScaler

# Parameters for FireBat and ARR-GA algorithms
ω1, ω2, ω3 = 0.4, 0.3, 0.3  # Weight parameters for fitness function (makespan, imbalance, utilization)
β0, γ, α = 1.0, 1.0, 0.1     # Firefly parameters (attractiveness, light absorption, stochastic factor)
f_min, f_max = 0.0, 2.0      # Frequency range for BAT algorithm
mutation_rate = 0.1           # Mutation rate for Genetic Algorithm
num_generations = 50          # Number of generations for ARR-GA
sim_config = Config(seed=1024, interval=1.0, step_size=0.01)
# Read the  task and VM data
task_data = pd.read_csv('task_data.csv')
vm_data = pd.read_csv('vm_data.csv')

# Normalize the VM resource data (CPU, Memory, I/O) for better comparison
scaler = MinMaxScaler()
vm_data[['CPU_Capacity', 'Memory_Capacity(MB)', 'IO_Capacity(MB/s)']] = scaler.fit_transform(
    vm_data[['CPU_Capacity', 'Memory_Capacity(MB)', 'IO_Capacity(MB/s)']])

# Function to calculate the Degree of Imbalance (DOI) - Equation 7
def calculate_DOI(vm_loads):
    avg_load = np.mean(vm_loads)
    doi = np.sqrt(np.mean((vm_loads - avg_load) ** 2))
    return doi

# Fitness function for task-VM mapping (Fi) - Equation 6
def fitness_function(vm_loads, makespan, utilization):
    DOI = calculate_DOI(vm_loads)
    fitness = (ω1 / makespan) + (ω2 / DOI) + (ω3 * utilization)
    print(f"Calculated Fitness: Makespan = {makespan}, DOI = {DOI}, Utilization = {utilization}, Fitness = {fitness}")
    return fitness

# Firefly Algorithm Phase - Initial exploration for optimal task-VM mappings
def firefly_algorithm(task_data, vm_data, num_fireflies=20, iterations=50):
    print("\n### Firefly Algorithm Phase ###")
    num_tasks = len(task_data)
    num_vms = len(vm_data)

    # Initialize fireflies (candidate solutions) as random task-to-VM mappings
    fireflies = [randint(0, num_vms, num_tasks) for _ in range(num_fireflies)]
    brightness = np.zeros(num_fireflies)

    for iteration in range(iterations):
        print(f"\n--- Iteration {iteration+1}/{iterations} ---")
        for i in range(num_fireflies):
            # Calculate fitness for each firefly based on task-VM mapping
            vm_loads = [sum(task_data[fireflies[i] == vm]['CPU_Requirement']) for vm in range(num_vms)]
            makespan = max(vm_loads)  # Makespan is the max load across all VMs
            utilization = sum(vm_loads) / (num_vms * max(vm_loads))  # Utilization is ratio of total load to max load
            brightness[i] = fitness_function(vm_loads, makespan, utilization)

        # Move fireflies towards brighter ones (Equation 8)
        for i in range(num_fireflies):
            for j in range(num_fireflies):
                if brightness[i] < brightness[j]:
                    distance = np.linalg.norm(fireflies[i] - fireflies[j])
                    # Update position (task-to-VM mapping) using floating-point values
                    fireflies[i] = fireflies[i] + β0 * np.exp(-γ * distance ** 2) * (fireflies[j] - fireflies[i]) + α * uniform(0, 1)
                    # Ensure that the firefly positions remain integers (valid VM indices)
                    fireflies[i] = np.round(fireflies[i]).astype(int)
                    fireflies[i] = np.clip(fireflies[i], 0, num_vms - 1)  # Ensure valid VM indices
                    print(f"Firefly {i} moved towards Firefly {j}")

    # Return the best mapping found
    best_firefly = np.argmax(brightness)
    print("\nBest solution found by Firefly Algorithm with brightness: ", brightness[best_firefly])
    return fireflies[best_firefly]

# BAT Algorithm Phase - Refining task-VM mappings
def bat_algorithm(task_data, vm_data, initial_mapping, num_bats=20, iterations=50):
    print("\n### BAT Algorithm Phase ###")
    num_tasks = len(task_data)
    num_vms = len(vm_data)

    # Initialize bats (candidate solutions) based on Firefly result
    bats = [np.copy(initial_mapping) for _ in range(num_bats)]
    velocities = [rand(num_tasks) for _ in range(num_bats)]
    frequencies = uniform(f_min, f_max, num_bats)
    best_solution = initial_mapping.copy()

    best_fitness = -np.inf
    for iteration in range(iterations):
        print(f"\n--- BAT Algorithm Iteration {iteration+1}/{iterations} ---")
        for i in range(num_bats):
            vm_loads = [sum(task_data[bats[i] == vm]['CPU_Requirement']) for vm in range(num_vms)]
            makespan = max(vm_loads)
            utilization = sum(vm_loads) / (num_vms * max(vm_loads))
            fitness = fitness_function(vm_loads, makespan, utilization)

            if fitness > best_fitness:
                best_fitness = fitness
                best_solution = bats[i].copy()
                print(f"New best solution found by Bat {i} with fitness: {best_fitness}")

            # Update velocity and position based on echolocation (Equations 9 and 10)
            velocities[i] = velocities[i] + (bats[i] - best_solution) * frequencies[i]
            bats[i] = bats[i] + velocities[i]
            bats[i] = np.clip(bats[i], 0, num_vms - 1)

    print("\nBest solution found by BAT Algorithm with fitness: ", best_fitness)
    return best_solution

# ARR-GA - Adaptive Resource Reallocation using Genetic Algorithm
def arr_ga(task_data, vm_data, initial_mapping, num_individuals=20, generations=num_generations):
    print("\n### ARR-GA Phase ###")
    num_tasks = len(task_data)
    num_vms = len(vm_data)

    # Initialize population based on preemption task allocations
    population = [np.copy(initial_mapping) for _ in range(num_individuals)]

    for generation in range(generations):
        print(f"\n--- ARR-GA Generation {generation+1}/{generations} ---")
        # Evaluate fitness for each individual
        fitness_scores = []
        for individual in population:
            vm_loads = [sum(task_data[individual == vm]['CPU_Requirement']) for vm in range(num_vms)]
            makespan = max(vm_loads)
            utilization = sum(vm_loads) / (num_vms * max(vm_loads))
            fitness = fitness_function(vm_loads, makespan, utilization)
            fitness_scores.append(fitness)

        # Selection (Tournament Selection)
        selected_individuals = []
        for _ in range(num_individuals):
            ind1, ind2 = randint(0, num_individuals), randint(0, num_individuals)
            selected_individuals.append(population[ind1] if fitness_scores[ind1] > fitness_scores[ind2] else population[ind2])

        # Crossover (Equation 14)
        next_population = []
        for i in range(0, num_individuals, 2):
            parent1, parent2 = selected_individuals[i], selected_individuals[i + 1]
            crossover_point = randint(0, num_tasks)
            child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
            next_population.extend([child1, child2])
            print(f"Crossover between individuals {i} and {i+1}")

        # Mutation (Equation 15)
        for individual in next_population:
            if rand() < mutation_rate:
                mutation_point = randint(0, num_tasks)
                individual[mutation_point] = randint(0, num_vms)  # Mutate a random task to a new VM
                print(f"Mutation applied at task {mutation_point}")

        population = next_population

    # Return the best solution found
    best_individual = np.argmax(fitness_scores)
    print("\nBest solution found by ARR-GA with fitness: ", fitness_scores[best_individual])
    return population[best_individual]

# Run FireBat (Firefly + BAT) and ARR-GA
print("\n### Running FireBat Algorithm ###")
firefly_solution = firefly_algorithm(task_data, vm_data)
bat_solution = bat_algorithm(task_data, vm_data, firefly_solution)

print("\n### Running ARR-GA for Final Reallocation ###")
final_solution = arr_ga(task_data, vm_data, bat_solution)

# Output the final task-to-VM mapping after ARR-GA optimization
task_data['Final_VM_ID'] = final_solution
print("\nFinal Task-to-VM Mapping:")
print(task_data[['Task_ID', 'Final_VM_ID']])
task_data.to_csv('final_task_to_vm_mapping.csv', index=False)


In [ ]:
import numpy as np
import pandas as pd
from numpy.random import rand, randint, uniform
from sklearn.preprocessing import MinMaxScaler

# Parameters for FireBat and ARR-GA algorithms
ω1, ω2, ω3 = 0.4, 0.3, 0.3  # Weight parameters for fitness function (makespan, imbalance, utilization)
β0, γ, α = 1.0, 1.0, 0.1     # Firefly parameters (attractiveness, light absorption, stochastic factor)
f_min, f_max = 0.0, 2.0      # Frequency range for BAT algorithm
mutation_rate = 0.1           # Mutation rate for Genetic Algorithm
num_generations = 50          # Number of generations for ARR-GA
task_migrations = 0           # Counter for task migrations

# Read the  task and VM data
task_data = pd.read_csv('task_data.csv')
vm_data = pd.read_csv('vm_data.csv')

# Normalize the VM resource data (CPU, Memory, I/O) for better comparison
scaler = MinMaxScaler()
vm_data[['CPU_Capacity', 'Memory_Capacity(MB)', 'IO_Capacity(MB/s)']] = scaler.fit_transform(
    vm_data[['CPU_Capacity', 'Memory_Capacity(MB)', 'IO_Capacity(MB/s)']])

# Function to calculate the Degree of Imbalance (DOI) - Equation 7
def calculate_DOI(vm_loads):
    avg_load = np.mean(vm_loads)
    doi = np.sqrt(np.mean((vm_loads - avg_load) ** 2))
    return doi

# Function to calculate Makespan - Equation 11
def calculate_makespan(vm_loads):
    return max(vm_loads)

# Function to calculate Response Time (assumed random start times for tasks)
def calculate_response_time(task_data):
    task_data['Start_Time'] = np.random.randint(0, 500, size=len(task_data))  # Assign random start times
    task_data['End_Time'] = task_data['Start_Time'] + task_data['Execution_Time(ms)']
    response_times = task_data['End_Time'] - task_data['Start_Time']
    return response_times.mean()

# Function to calculate Throughput - Equation 12
def calculate_throughput(num_tasks, total_time):
    return num_tasks / total_time

# Function to calculate CPU Utilization
def calculate_cpu_utilization(vm_loads, vm_data):
    total_cpu = sum(vm_data['CPU_Capacity'])
    used_cpu = sum(vm_loads)
    return (used_cpu / total_cpu) * 100

# Fitness function for task-VM mapping (Fi) - Equation 6
def fitness_function(vm_loads, makespan, utilization):
    DOI = calculate_DOI(vm_loads)
    fitness = (ω1 / makespan) + (ω2 / DOI) + (ω3 * utilization)
    print(f"Calculated Fitness: Makespan = {makespan}, DOI = {DOI}, Utilization = {utilization}, Fitness = {fitness}")
    return fitness

# Firefly Algorithm Phase - Initial exploration for optimal task-VM mappings
def firefly_algorithm(task_data, vm_data, num_fireflies=20, iterations=50):
    print("\n### Firefly Algorithm Phase ###")
    num_tasks = len(task_data)
    num_vms = len(vm_data)

    # Initialize fireflies (candidate solutions) as random task-to-VM mappings
    fireflies = [randint(0, num_vms, num_tasks) for _ in range(num_fireflies)]
    brightness = np.zeros(num_fireflies)

    for iteration in range(iterations):
        print(f"\n--- Iteration {iteration+1}/{iterations} ---")
        for i in range(num_fireflies):
            # Calculate fitness for each firefly based on task-VM mapping
            vm_loads = [sum(task_data[fireflies[i] == vm]['CPU_Requirement']) for vm in range(num_vms)]
            makespan = calculate_makespan(vm_loads)  # Makespan is the max load across all VMs
            utilization = sum(vm_loads) / (num_vms * max(vm_loads))  # Utilization is ratio of total load to max load
            brightness[i] = fitness_function(vm_loads, makespan, utilization)

        # Move fireflies towards brighter ones (Equation 8)
        for i in range(num_fireflies):
            for j in range(num_fireflies):
                if brightness[i] < brightness[j]:
                    distance = np.linalg.norm(fireflies[i] - fireflies[j])
                    # Update position (task-to-VM mapping) using floating-point values
                    fireflies[i] = fireflies[i] + β0 * np.exp(-γ * distance ** 2) * (fireflies[j] - fireflies[i]) + α * uniform(0, 1)
                    # Ensure that the firefly positions remain integers (valid VM indices)
                    fireflies[i] = np.round(fireflies[i]).astype(int)
                    fireflies[i] = np.clip(fireflies[i], 0, num_vms - 1)  # Ensure valid VM indices
                    print(f"Firefly {i} moved towards Firefly {j}")

    # Return the best mapping found
    best_firefly = np.argmax(brightness)
    print("\nBest solution found by Firefly Algorithm with brightness: ", brightness[best_firefly])
    return fireflies[best_firefly]

# BAT Algorithm Phase - Refining task-VM mappings
def bat_algorithm(task_data, vm_data, initial_mapping, num_bats=20, iterations=50):
    print("\n### BAT Algorithm Phase ###")
    num_tasks = len(task_data)
    num_vms = len(vm_data)

    # Initialize bats (candidate solutions) based on Firefly result
    bats = [np.copy(initial_mapping) for _ in range(num_bats)]
    velocities = [rand(num_tasks) for _ in range(num_bats)]
    frequencies = uniform(f_min, f_max, num_bats)
    best_solution = initial_mapping.copy()

    best_fitness = -np.inf
    for iteration in range(iterations):
        print(f"\n--- BAT Algorithm Iteration {iteration+1}/{iterations} ---")
        for i in range(num_bats):
            vm_loads = [sum(task_data[bats[i] == vm]['CPU_Requirement']) for vm in range(num_vms)]
            makespan = calculate_makespan(vm_loads)
            utilization = sum(vm_loads) / (num_vms * max(vm_loads))
            fitness = fitness_function(vm_loads, makespan, utilization)

            if fitness > best_fitness:
                best_fitness = fitness
                best_solution = bats[i].copy()
                print(f"New best solution found by Bat {i} with fitness: {best_fitness}")

            # Update velocity and position based on echolocation (Equations 9 and 10)
            velocities[i] = velocities[i] + (bats[i] - best_solution) * frequencies[i]
            bats[i] = bats[i] + velocities[i]
            bats[i] = np.clip(bats[i], 0, num_vms - 1)

    print("\nBest solution found by BAT Algorithm with fitness: ", best_fitness)
    return best_solution

# ARR-GA - Adaptive Resource Reallocation using Genetic Algorithm
def arr_ga(task_data, vm_data, initial_mapping, num_individuals=20, generations=num_generations):
    global task_migrations
    print("\n### ARR-GA Phase ###")
    num_tasks = len(task_data)
    num_vms = len(vm_data)

    # Initialize population based on preemption task allocations
    population = [np.copy(initial_mapping) for _ in range(num_individuals)]

    for generation in range(generations):
        print(f"\n--- ARR-GA Generation {generation+1}/{generations} ---")
        # Evaluate fitness for each individual
        fitness_scores = []
        for individual in population:
            vm_loads = [sum(task_data[individual == vm]['CPU_Requirement']) for vm in range(num_vms)]
            makespan = calculate_makespan(vm_loads)
            utilization = sum(vm_loads) / (num_vms * max(vm_loads))
            fitness = fitness_function(vm_loads, makespan, utilization)
            fitness_scores.append(fitness)

        # Selection (Tournament Selection)
        selected_individuals = []
        for _ in range(num_individuals):
            ind1, ind2 = randint(0, num_individuals), randint(0, num_individuals)
            selected_individuals.append(population[ind1] if fitness_scores[ind1] > fitness_scores[ind2] else population[ind2])

        # Crossover (Equation 14)
        next_population = []
        for i in range(0, num_individuals, 2):
            parent1, parent2 = selected_individuals[i], selected_individuals[i + 1]
            crossover_point = randint(0, num_tasks)
            child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
            next_population.extend([child1, child2])
            print(f"Crossover between individuals {i} and {i+1}")

        # Mutation (Equation 15)
        for individual in next_population:
            if rand() < mutation_rate:
                mutation_point = randint(0, num_tasks)
                individual[mutation_point] = randint(0, num_vms)  # Mutate a random task to a new VM
                task_migrations += 1
                print(f"Mutation applied at task {mutation_point}")

        population = next_population

    # Return the best solution found
    best_individual = np.argmax(fitness_scores)
    print("\nBest solution found by ARR-GA with fitness: ", fitness_scores[best_individual])
    return population[best_individual]

# Run FireBat (Firefly + BAT) and ARR-GA
print("\n### Running FireBat Algorithm ###")
firefly_solution = firefly_algorithm(task_data, vm_data)
bat_solution = bat_algorithm(task_data, vm_data, firefly_solution)

print("\n### Running ARR-GA for Final Reallocation ###")
final_solution = arr_ga(task_data, vm_data, bat_solution)

# Output the final task-to-VM mapping after ARR-GA optimization
task_data['Final_VM_ID'] = final_solution
print("\nFinal Task-to-VM Mapping:")
print(task_data[['Task_ID', 'Final_VM_ID']])

# Estimate Makespan, DOI, Response Time, Throughput, CPU Utilization
num_tasks = len(task_data)
vm_loads = [sum(task_data[final_solution == vm]['CPU_Requirement']) for vm in range(len(vm_data))]

makespan = calculate_makespan(vm_loads)
doi = calculate_DOI(vm_loads)
response_time = calculate_response_time(task_data)
throughput = calculate_throughput(num_tasks, makespan)
cpu_utilization = calculate_cpu_utilization(vm_loads, vm_data)

# Display results
print("\n### Optimization Results ###")
print(f"Makespan: {makespan}")
print(f"Degree of Imbalance (DOI): {doi}")
print(f"Average Response Time: {response_time}")
print(f"Throughput: {throughput} tasks per unit time")
print(f"CPU Utilization: {cpu_utilization:.2f}%")
print(f"Task Migrations (per {num_tasks} tasks): {task_migrations}")

# Save the final task-to-VM mapping to a CSV file
task_data.to_csv('final_task_to_vm_mapping.csv', index=False)


Streaming output truncated to the last 5000 lines.
Firefly 15 moved towards Firefly 2
Firefly 15 moved towards Firefly 3
Firefly 15 moved towards Firefly 4
Firefly 15 moved towards Firefly 5
Firefly 15 moved towards Firefly 9
Firefly 15 moved towards Firefly 10
Firefly 15 moved towards Firefly 11
Firefly 15 moved towards Firefly 17
Firefly 16 moved towards Firefly 1
Firefly 16 moved towards Firefly 2
Firefly 16 moved towards Firefly 3
Firefly 16 moved towards Firefly 4
Firefly 16 moved towards Firefly 5
Firefly 16 moved towards Firefly 9
Firefly 16 moved towards Firefly 10
Firefly 16 moved towards Firefly 11
Firefly 16 moved towards Firefly 15
Firefly 16 moved towards Firefly 17
Firefly 17 moved towards Firefly 3
Firefly 17 moved towards Firefly 4
Firefly 18 moved towards Firefly 0
Firefly 18 moved towards Firefly 1
Firefly 18 moved towards Firefly 2
Firefly 18 moved towards Firefly 3
Firefly 18 moved towards Firefly 4
Firefly 18 moved towards Firefly 5
Firefly 18 moved towards Firefly